In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient

In [23]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
# connect to mongo db
client = MongoClient('mongodb://localhost:27017/')
db = client.pokemon_db
# get pokemon base data collection
col_pokemon = db['pokemon_move']
pokemon_base = pd.DataFrame(list(col_pokemon.find())).drop(columns=['_id'])

In [3]:
# get pokemon raw data collection
col_raw = db['pokemon_data']
pokemon_raw = pd.DataFrame(list(col_raw.find())).drop(columns=['_id'])

In [4]:
pokemon_elv = pokemon_base.loc[pokemon_base.evol_id != -1]

In [35]:
pokemon_base

,area,avg_acc_power,base_experience,catch_rate_avg,chance_avg,encount_rate_avg,evol_id,evolution,height,move_name,...,stat_defense,stat_hp,stat_special-attack,stat_special-defense,stat_speed,stat_total_attack,stat_total_defense,status_rate,type,weight
0,NaN,31.29,64,NaN,NaN,NaN,0.0,3-1,7,"[razor-wind, swords-dance, cut, bind, vine-whi...",...,49,45,65,65,45,114,114,46.15,poison,69
1,NaN,27.68,142,NaN,NaN,NaN,0.0,3-2,10,"[swords-dance, cut, bind, vine-whip, headbutt,...",...,63,60,80,80,60,142,143,46.97,poison,130
2,NaN,36.34,236,NaN,NaN,NaN,0.0,3-3,20,"[swords-dance, cut, bind, vine-whip, headbutt,...",...,83,80,100,100,80,182,183,42.86,poison,1000
3,NaN,41.89,62,NaN,NaN,NaN,1.0,3-1,6,"[mega-punch, fire-punch, thunder-punch, scratc...",...,43,39,60,50,65,112,93,27.96,fire,85
4,NaN,40.99,142,NaN,NaN,NaN,1.0,3-2,11,"[mega-punch, fire-punch, thunder-punch, scratc...",...,58,58,80,65,80,144,123,28.57,fire,190
5,NaN,45.77,240,NaN,NaN,NaN,1.0,3-3,17,"[mega-punch, fire-punch, thunder-punch, scratc...",...,78,78,109,85,100,193,163,26.85,flying,905
6,NaN,38.84,63,NaN,NaN,NaN,2.0,3-1,5,"[mega-punch, ice-punch, mega-kick, headbutt, t...",...,65,44,50,64,43,98,129,30.77,water,90
7,NaN,41.81,142,NaN,NaN,NaN,2.0,3-2,10,"[mega-punch, ice-punch, mega-kick, headbutt, t...",...,80,59,65,80,58,128,160,27.63,water,225
8,NaN,48.27,239,NaN,NaN,NaN,2.0,3-3,16,"[mega-punch, ice-punch, mega-kick, headbutt, t...",...,100,79,85,105,78,168,205,23.66,water,855
9,16.0,40.45,39,2.353125,8.35,1.34,3.0,3-1,3,"[tackle, string-shot, snore, bug-bite, electro...",...,35,45,20,20,45,50,55,20.00,bug,29


In [8]:
pokemon_base.groupby('evolution').mean().drop(columns=['evol_id'])

,area,avg_acc_power,base_experience,catch_rate_avg,chance_avg,encount_rate_avg,height,move_num,physical_rate,special_rate,...,stat_avg,stat_defense,stat_hp,stat_special-attack,stat_special-defense,stat_speed,stat_total_attack,stat_total_defense,status_rate,weight
evolution,,,,,,,,,,,,,,,,,,,,,
0-0,11.859649,37.512121,212.689394,1.064247,9.282807,1.206491,16.871212,70.962121,35.247879,25.252500,...,87.279091,90.704545,82.219697,86.636364,90.439394,81.030303,179.280303,181.143939,39.498939,1188.636364
2-1,15.120000,32.796278,63.583333,2.421085,12.087933,2.641667,6.616667,65.750000,35.942944,22.411778,...,52.428667,54.477778,51.544444,48.472222,51.033333,52.522222,104.994444,105.511111,41.645500,211.166667
2-2,10.692308,38.738523,167.812500,1.728700,8.450769,1.129060,15.426136,68.488636,38.495000,24.175057,...,79.902102,81.539773,79.960227,77.977273,78.477273,73.119318,166.318182,160.017045,37.330398,833.306818
3-1,13.625000,32.331183,57.870968,1.967741,10.295938,1.893437,5.032258,61.849462,35.409892,22.560000,...,47.427957,48.225806,46.817204,47.387097,45.709677,44.677419,99.139785,93.935484,42.030645,122.086022
3-2,14.169492,34.449892,138.451613,1.266453,8.339153,1.499492,9.376344,62.462366,36.538495,23.234194,...,64.641613,67.806452,64.903226,64.516129,63.440860,58.731183,132.967742,131.247312,40.227634,437.043011
3-3,3.551724,40.802174,236.793478,0.145172,3.305172,0.121724,15.652174,74.369565,39.591739,25.141087,...,86.586522,83.576087,85.750000,91.086957,85.358696,78.347826,186.489130,168.934783,35.267500,921.663043
n.a,10.454545,38.089596,199.732323,1.591634,9.933636,2.270000,15.671717,60.535354,35.119192,25.097626,...,85.438990,85.661616,70.686869,89.580808,85.035354,85.686869,185.565657,170.696970,39.783081,880.803030


In [14]:
evl_2 = pokemon_base.loc[pokemon_base.evolution == '2-2'].append(pokemon_base.loc[pokemon_base.evolution == '2-1'])

In [4]:
# some pokemon are multi-types
pokemon_type_df = pokemon_raw[['name', 'types']]

In [5]:
pokemon_type_list = []
for index, row in pokemon_type_df.iterrows():
    name = row[0]
    type_list = row[1]
    if len(type_list) > 1:
        pokemon = {'name': name, 'type_1': type_list[0]['type']['name'], 'type_2': type_list[1]['type']['name']}
    else:
        pokemon = {'name': name, 'type_1': type_list[0]['type']['name'], 'type_2': 'n.a'}
    
    pokemon_type_list.append(pokemon)

In [6]:
pokemon_type_df = pd.DataFrame(pokemon_type_list)

In [7]:
pokemon_base_new = pokemon_base.merge(pokemon_type_df, on='name', how='left')

In [17]:
# sort types
pokemon_type_sort_list = []
for index, row in pokemon_base_new[['name', 'type_1', 'type_2']].iterrows():
    name = row[0]
    type_1 = row[1]
    type_2 = row[2]
    type_list = [type_1, type_2]
    if type_list[1] != 'n.a':
        type_list.sort()
        type_output = type_list[0] + '&' + type_list[1]
    else:
        type_output = type_list[0]
        
    pokemon_type_sort_list.append({'name': name, 'types': type_output})
    
pokemon_type_sort_df = pd.DataFrame(pokemon_type_sort_list)

In [20]:
pokemon_base_new_types = pokemon_base_new.merge(pokemon_type_sort_df, on='name', how='left').drop(columns=['type'])

In [26]:
pokemon_base_new_types.groupby('types').avg_acc_power.mean().sort_values(ascending=False)

types
fire&water         58.490000
dragon&fire        50.786667
fire&steel         48.900000
dark&rock          48.870000
flying             48.630000
dragon&poison      48.495000
dragon&ice         48.406667
dragon&flying      48.047500
dragon&grass       47.700000
ground&poison      47.645000
electric&rock      46.573333
dark&dragon        46.522500
dragon&normal      46.440000
dragon&rock        45.615000
fighting&fire      45.360000
dragon&ground      45.287000
ground&normal      45.210000
bug&fire           45.090000
fire&psychic       44.583333
dragon&water       44.445000
dragon&electric    44.435000
dark&fighting      44.396667
dragon&ghost       44.080000
fire&ground        43.932500
dragon&steel       43.540000
ground&water       43.463000
fighting&water     43.373333
ground&rock        42.756667
flying&ground      42.715000
dragon&fairy       42.640000
                     ...    
flying&normal      32.160370
dark&psychic       32.080000
flying&psychic     31.807143
fairy&ic

In [34]:
pokemon_base_new_types.groupby('evolution').types.value_counts()

evolution  types           
0-0        normal              17
           psychic              8
           electric             6
           water                6
           bug                  3
           bug&fighting         3
           fairy&steel          3
           fighting             3
           fire                 3
           bug&steel            2
           dark                 2
           dark&ghost           2
           dragon&fire          2
           dragon&psychic       2
           electric&fairy       2
           electric&flying      2
           fairy                2
           fairy&rock           2
           fire&flying          2
           flying&ice           2
           flying&normal        2
           flying&psychic       2
           flying&steel         2
           grass                2
           ice                  2
           normal&psychic       2
           psychic&rock         2
           rock&water           2
           bug&rock 